In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/System-Threat-Forecaster/sample_submission.csv
/kaggle/input/System-Threat-Forecaster/train.csv
/kaggle/input/System-Threat-Forecaster/test.csv


In [41]:
train = pd.read_csv("/kaggle/input/System-Threat-Forecaster/train.csv")
test = pd.read_csv("/kaggle/input/System-Threat-Forecaster/test.csv")


In [3]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler


In [4]:
train.columns

Index(['MachineID', 'ProductName', 'EngineVersion', 'AppVersion',
       'SignatureVersion', 'IsBetaUser', 'RealTimeProtectionState',
       'IsPassiveModeEnabled', 'AntivirusConfigID',
       'NumAntivirusProductsInstalled', 'NumAntivirusProductsEnabled',
       'HasTpm', 'CountryID', 'CityID', 'GeoRegionID', 'LocaleEnglishNameID',
       'PlatformType', 'Processor', 'OSVersion', 'OSBuildNumber',
       'OSProductSuite', 'OsPlatformSubRelease', 'OSBuildLab',
       'SKUEditionName', 'IsSystemProtected', 'AutoSampleSubmissionEnabled',
       'SMode', 'IEVersionID', 'FirewallEnabled', 'EnableLUA',
       'MDC2FormFactor', 'DeviceFamily', 'OEMNameID', 'OEMModelID',
       'ProcessorCoreCount', 'ProcessorManufacturerID', 'ProcessorModelID',
       'PrimaryDiskCapacityMB', 'PrimaryDiskType', 'SystemVolumeCapacityMB',
       'HasOpticalDiskDrive', 'TotalPhysicalRAMMB', 'ChassisType',
       'PrimaryDisplayDiagonalInches', 'PrimaryDisplayResolutionHorizontal',
       'PrimaryDisplayResolutio

In [5]:
train.shape

(100000, 76)

In [6]:
test.shape

(10000, 75)

There are 1 lakh rows and 76 columns are there in the train dataset and there are ten thousand rows and 75 columns are there in the test dataset.

# Explanatory Data Analysis

In [12]:
train.shape

(100000, 76)

There are 43 columns having missing values in the whole dataset.

In [14]:
one_unique_value = train.columns[train.nunique()==1]
print(one_unique_value)

Index(['IsBetaUser', 'AutoSampleSubmissionEnabled', 'IsFlightsDisabled'], dtype='object')


These columns contain only one unique value which doesn't makes much sense so i remove these columns.

# Preprocessing

In [4]:
train['ProductName'] = train['ProductName'].replace({'win8defender': 1, 'mse': 0})
train['DeviceFamily'] = train['DeviceFamily'].replace({'Windows.Server':0, 'Windows.Desktop':1})

<ipython-input-4-eb3d19d47d4b>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train['ProductName'] = train['ProductName'].replace({'win8defender': 1, 'mse': 0})
<ipython-input-4-eb3d19d47d4b>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train['DeviceFamily'] = train['DeviceFamily'].replace({'Windows.Server':0, 'Windows.Desktop':1})


In [5]:
columns_to_fill = ['CityID', 'NumAntivirusProductsEnabled']
train[columns_to_fill] = train[columns_to_fill].fillna(0)

In [6]:
train['PrimaryDiskType'] = train['PrimaryDiskType'].replace(['UNKNOWN','Unknown','Unspecified'], 'Unknown')
train['PrimaryDiskType'] = train['PrimaryDiskType'].fillna('Unknown')



'DateOS','DateAS','FirmwareVersionID' , 'SystemVolumeCapacityMB' , 'OEMModelID' , 'CityID' , 'MachineID'. They contain lot of unique values





In [7]:
train = train.drop(columns= ['IsBetaUser','DateAS' 'AutoSampleSubmissionEnabled', 'IsFlightsDisabled','DateOS','FirmwareVersionID' , 'SystemVolumeCapacityMB' , 'OEMModelID' , 'CityID' , 'MachineID'], errors='ignore')

In [9]:
numeric_cols = train.select_dtypes(include=['int']).columns.tolist()
object_cols = train.select_dtypes(include=['object']).columns.tolist()
float_cols = train.select_dtypes(include=['float64']).columns.tolist()

print("\nNumeric:", len(numeric_cols), numeric_cols)
print("\nObject:", len(object_cols), object_cols)
print("\nFloat:", len(float_cols), float_cols)


Numeric: 18 ['ProductName', 'IsPassiveModeEnabled', 'HasTpm', 'CountryID', 'LocaleEnglishNameID', 'OSBuildNumber', 'OSProductSuite', 'AutoSampleSubmissionEnabled', 'DeviceFamily', 'HasOpticalDiskDrive', 'OSBuildNumberOnly', 'OSBuildRevisionOnly', 'OSUILocaleID', 'IsPortableOS', 'IsSecureBootEnabled', 'IsTouchEnabled', 'IsPenCapable', 'target']

Object: 24 ['EngineVersion', 'AppVersion', 'SignatureVersion', 'PlatformType', 'Processor', 'OSVersion', 'OsPlatformSubRelease', 'OSBuildLab', 'SKUEditionName', 'MDC2FormFactor', 'PrimaryDiskType', 'ChassisType', 'PowerPlatformRole', 'NumericOSVersion', 'OSArchitecture', 'OSBranch', 'OSEdition', 'OSSkuFriendlyName', 'OSInstallType', 'AutoUpdateOptionsName', 'OSGenuineState', 'LicenseActivationChannel', 'FlightRing', 'DateAS']

Float: 26 ['RealTimeProtectionState', 'AntivirusConfigID', 'NumAntivirusProductsInstalled', 'NumAntivirusProductsEnabled', 'GeoRegionID', 'IsSystemProtected', 'SMode', 'IEVersionID', 'FirewallEnabled', 'EnableLUA', 'OEMNa

# Simple Imputer

In [10]:
train = train.replace({pd.NA: np.nan, '': np.nan, None: np.nan})
all_columns = {"numeric": numeric_cols, "object": object_cols, "float": float_cols}
mode_imputer = SimpleImputer(strategy='most_frequent')

In [11]:
# Impute each column type
for col_type, columns in all_columns.items():
    if columns:  # Ensure there are columns to process
        train[columns] = pd.DataFrame(
            mode_imputer.fit_transform(train[columns]),
            columns=columns,
            index=train.index
        )

In [12]:
for col in all_columns["numeric"]:
    train[col] = train[col].astype(int)
for col in all_columns["float"]:
    train[col] = train[col].astype(float)
for col in all_columns["object"]:
    train[col] = train[col].astype(str)
print(train.isnull().sum())

ProductName                         0
EngineVersion                       0
AppVersion                          0
SignatureVersion                    0
RealTimeProtectionState             0
                                   ..
IsAlwaysOnAlwaysConnectedCapable    0
IsGamer                             0
RegionIdentifier                    0
DateAS                              0
target                              0
Length: 68, dtype: int64


In [13]:
unique_counts = train.nunique()
binary_cols = unique_counts[unique_counts == 2].index
binary = train[binary_cols]
print(binary.head())


   ProductName  IsPassiveModeEnabled  HasTpm  IsSystemProtected  SMode  \
0            1                     0       1                1.0    0.0   
1            1                     0       1                1.0    0.0   
2            1                     0       1                1.0    0.0   
3            1                     0       1                1.0    0.0   
4            1                     0       1                1.0    0.0   

   FirewallEnabled  DeviceFamily  HasOpticalDiskDrive  IsPortableOS  \
0              1.0             1                    0             0   
1              1.0             1                    1             0   
2              1.0             1                    1             0   
3              1.0             1                    0             0   
4              1.0             1                    0             0   

   IsSecureBootEnabled  IsVirtualDevice  IsTouchEnabled  IsPenCapable  \
0                    0              0.0               1

In [14]:
# Select columns with int64 dtype
int_cols = train.select_dtypes(include=['int64']).columns.tolist()

# Convert int64 columns to float64
for col in int_cols:
    train[col] = train[col].astype('float64', errors='ignore')

# Print the updated info
print(train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 68 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   ProductName                         100000 non-null  float64
 1   EngineVersion                       100000 non-null  object 
 2   AppVersion                          100000 non-null  object 
 3   SignatureVersion                    100000 non-null  object 
 4   RealTimeProtectionState             100000 non-null  float64
 5   IsPassiveModeEnabled                100000 non-null  float64
 6   AntivirusConfigID                   100000 non-null  float64
 7   NumAntivirusProductsInstalled       100000 non-null  float64
 8   NumAntivirusProductsEnabled         100000 non-null  float64
 9   HasTpm                              100000 non-null  float64
 10  CountryID                           100000 non-null  float64
 11  GeoRegionID                

# Train-Test Split

In [14]:
y= train.target.copy()
x= train.drop('target', axis=1).copy()

In [15]:
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val = train_test_split(x, y, test_size=0.15, random_state=1)

In [16]:
X_train.shape


(85000, 67)

In [20]:
X_val.shape

(15000, 67)

In [35]:
y_train.shape

(85000,)

In [36]:
y_val.shape

(15000,)

# Pipeline

In [17]:
from sklearn.compose import make_column_selector as mcs
num_sel = mcs(dtype_exclude='object')
cat_sel = mcs(dtype_include='object')

In [18]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, FunctionTransformer,OneHotEncoder
from sklearn.compose import ColumnTransformer

# Create numerical and categorical pipelines
num_pipeline = Pipeline([
    ('scaler', StandardScaler())
])

# Custom transformer to convert float outputs to integers
def convert_to_int(X):
    return X.astype(np.int64)  # Convert to integer

cat_pipeline = Pipeline([
    ('ordinal', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)),
    ('convert_to_int', FunctionTransformer(convert_to_int))  # Convert floats to integers
])

# Combine numerical and categorical pipelines using ColumnTransformer
preprocessor = ColumnTransformer([
    ('numerical', num_pipeline, num_sel(X_train)),
    ('categorical', cat_pipeline, cat_sel(X_train))
])

# Create and fit the complete pipeline
pipe = Pipeline([('preprocessor', preprocessor)])
pipe.fit(X_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['ProductName',
                                                   'RealTimeProtectionState',
                                                   'IsPassiveModeEnabled',
                                                   'AntivirusConfigID',
                                                   'NumAntivirusProductsInstalled',
                                                   'NumAntivirusProductsEnabled',
                                                   'HasTpm', 'CountryID',
                                                   'GeoRegionID',
                                                   'LocaleEnglishNameID',
                                                   'OSBuildNumber',
                                                   'OSProdu...
                                                   'PlatformType', 'Processor',
                                                   'OSVersion',
                                                   'OsPlatformSubRelease',
                                                   'OSBuildLab',
                                                   'SKUEditionName',
                                                   'MDC2FormFactor',
                                                   'PrimaryDiskType',
                                                   'ChassisType',
                                                   'PowerPlatformRole',
                                                   'NumericOSVersion',
                                                   'OSArchitecture', 'OSBranch',
                                                   'OSEdition',
                                                   'OSSkuFriendlyName',
                                                   'OSInstallType',
                                                   'AutoUpdateOptionsName',
                                                   'OSGenuineState',
                                                   'LicenseActivationChannel',
                                                   'FlightRing',
                                                   'DateAS'])]))])

In [19]:
X_train_trf = pipe.transform(X_train)
X_val_trf = pipe.transform(X_val)

In [20]:
X_train_trf.shape

(85000, 67)

#Selector

In [22]:
from sklearn.feature_selection import SelectKBest, mutual_info_classif, f_classif
SelectKBest_selector = SelectKBest(mutual_info_classif, k=67)
SelectKBest_selector.fit(X_train_trf, y_train)

SelectKBest(k=67, score_func=<function mutual_info_classif at 0x78d7e1682560>)

In [23]:
X_train_selected = SelectKBest_selector.transform(X_train_trf)
X_val_selected = SelectKBest_selector.transform(X_val_trf)

In [24]:
def selector_method(selector, X_train_trf, y_train):
    selector.fit(X_train_trf, y_train)
    X_train_selected = selector.transform(X_train_trf)
    X_val_selected = selector.transform(X_val_trf)
    return X_train_selected, X_val_selected

# DummyClassifier

In [25]:
# baseline model
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score

# Dummy Classifier with default strategy
dummy_clf = DummyClassifier(random_state=42)
dummy_clf.fit(X_train_selected, y_train)

# Compute accuracy for training data
train_accuracy = accuracy_score(y_train, dummy_clf.predict(X_train))
print(f"Baseline Model Train Accuracy: {train_accuracy}")

Baseline Model Train Accuracy: 0.5065764705882353


In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

log_reg = LogisticRegression()
log_reg.fit(X_train_selected, y_train)

y_pred_log = log_reg.predict(X_val_selected)
log_accuracy = accuracy_score(y_val, y_pred_log)
print(f"Logistic Regression Accuracy: {log_accuracy:.4f}")


Logistic Regression Accuracy: 0.5851


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Hyperparameter Tuning
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100], 
    'penalty': ['l1', 'l2', 'elasticnet'], 
    'solver': ['saga']
}

log_reg = GridSearchCV(LogisticRegression(max_iter=1000), param_grid, cv=5, scoring='accuracy', n_jobs=-1)
log_reg.fit(X_train_selected, y_train)

# Step 4: Evaluate the best model
best_model = log_reg.best_estimator_
y_pred_log = best_model.predict(X_val_selected)

log_accuracy = accuracy_score(y_val, y_pred_log)
print(f"Optimized Logistic Regression Accuracy: {log_accuracy:.4f}")
print("\nClassification Report:\n", classification_report(y_val, y_pred_log))

# Print best hyperparameters
print("Best Hyperparameters:", log_reg.best_params_)


In [27]:
from sklearn.linear_model import RidgeClassifier

ridge_clf = RidgeClassifier(alpha=1.0)
ridge_clf.fit(X_train_selected, y_train)

y_pred_ridge = ridge_clf.predict(X_val_selected)
ridge_accuracy = accuracy_score(y_val, y_pred_ridge)
print(f"Ridge Classifier Accuracy: {ridge_accuracy:.4f}")


Ridge Classifier Accuracy: 0.5885


In [28]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_train_selected, y_train)

y_pred = rf_clf.predict(X_val_selected)
print(f"Random Forest Accuracy: {accuracy_score(y_val, y_pred):.4f}")


Random Forest Accuracy: 0.6153


In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE



# Define parameter grid for hyperparameter tuning
param_grid = {
    'n_estimators': [100, 300, 500],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

# Perform grid search with cross-validation
grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=3, n_jobs=-1, verbose=1)
grid_search.fit(X_train_selected, y_train)

# Get the best model
best_rf_clf = grid_search.best_estimator_

# Make predictions
y_pred = best_rf_clf.predict(X_val_selected)

# Evaluate the model
accuracy = accuracy_score(y_val, y_pred)
print(f"Optimized Random Forest Accuracy: {accuracy:.4f}")
print(classification_report(y_val, y_pred))


Fitting 3 folds for each of 162 candidates, totalling 486 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


In [29]:
from xgboost import XGBClassifier

xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_clf.fit(X_train_selected, y_train)

y_pred = xgb_clf.predict(X_val_selected)
print(f"XGBoost Accuracy: {accuracy_score(y_val, y_pred):.4f}")


XGBoost Accuracy: 0.6143


In [31]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

param_dist = {
    'n_estimators': np.arange(100, 500, 100),  # Random search over range
    'max_depth': np.arange(3, 10, 2),
    'learning_rate': np.linspace(0.01, 0.2, 5),
    'subsample': [0.7, 0.8, 1.0],
    'colsample_bytree': [0.7, 0.8, 1.0],
    'gamma': [0, 0.1, 0.2],
    'reg_alpha': [0, 0.1, 0.5],
    'reg_lambda': [1, 1.5, 2.0]
}

xgb_clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss', tree_method='hist')

random_search_xgb = RandomizedSearchCV(
    xgb_clf, param_dist, n_iter=30,  # Searches only 30 random combinations
    scoring='accuracy', cv=3, n_jobs=-1, verbose=1, random_state=42
)

random_search_xgb.fit(X_train_selected, y_train)

print("Best parameters:", random_search_xgb.best_params_)
print(f"Best accuracy: {random_search_xgb.best_score_:.4f}")



Fitting 3 folds for each of 30 candidates, totalling 90 fits
Best parameters: {'subsample': 1.0, 'reg_lambda': 1, 'reg_alpha': 0.1, 'n_estimators': 400, 'max_depth': 5, 'learning_rate': 0.0575, 'gamma': 0.2, 'colsample_bytree': 1.0}
Best accuracy: 0.6274


In [30]:
from lightgbm import LGBMClassifier

lgbm_clf = LGBMClassifier()
lgbm_clf.fit(X_train_selected, y_train)

y_pred = lgbm_clf.predict(X_val_selected)
print(f"LightGBM Accuracy: {accuracy_score(y_val, y_pred):.4f}")


[LightGBM] [Info] Number of positive: 43059, number of negative: 41941
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.027549 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3679
[LightGBM] [Info] Number of data points in the train set: 85000, number of used features: 66
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.506576 -> initscore=0.026307
[LightGBM] [Info] Start training from score 0.026307
LightGBM Accuracy: 0.6216


In [32]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# Define hyperparameter space
param_dist = {
    'num_leaves': np.arange(20, 150, 10),  # Controls model complexity
    'learning_rate': [0.01, 0.05, 0.1, 0.2],  # Lower values for stable training
    'n_estimators': [100, 300, 500, 700],  # Number of boosting rounds
    'max_depth': [-1, 5, 10, 15],  # Depth of each tree
    'min_child_samples': [10, 20, 30, 50],  # Minimum samples per leaf
    'subsample': [0.6, 0.8, 1.0],  # Row sampling
    'colsample_bytree': [0.6, 0.8, 1.0]  # Feature sampling
}

# Use RandomizedSearchCV for efficiency
random_search = RandomizedSearchCV(
    LGBMClassifier(random_state=42),
    param_distributions=param_dist,
    n_iter=20,  # Adjustable, more iterations mean better tuning
    cv=3,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

# Fit the model
random_search.fit(X_train_selected, y_train)

# Get the best model
best_lgbm_clf = random_search.best_estimator_

# Make predictions
y_pred = best_lgbm_clf.predict(X_val_selected)

# Evaluate performance
accuracy = accuracy_score(y_val, y_pred)
print(f"Optimized LightGBM Accuracy: {accuracy:.4f}")


Fitting 3 folds for each of 20 candidates, totalling 60 fits
[LightGBM] [Info] Number of positive: 43059, number of negative: 41941
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.047136 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3676
[LightGBM] [Info] Number of data points in the train set: 85000, number of used features: 65
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.506576 -> initscore=0.026307
[LightGBM] [Info] Start training from score 0.026307
Optimized LightGBM Accuracy: 0.6291


In [ ]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 300, 500, 700],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7, 10],
    'num_leaves': [20, 31, 40, 50],
    'min_child_samples': [10, 20, 30],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

# Initialize LightGBM
lgbm_clf = LGBMClassifier()

# Use RandomizedSearchCV for hyperparameter tuning
random_search = RandomizedSearchCV(
    estimator=lgbm_clf,
    param_distributions=param_grid,
    n_iter=20,  # Number of random samples
    scoring='accuracy',
    cv=5,  # 5-Fold Cross Validation
    verbose=1,
    n_jobs=-1
)

# Fit on training data
random_search.fit(X_train_selected, y_train)

# Get the best model
best_lgbm = random_search.best_estimator_

# Predict on validation set
y_pred = best_lgbm.predict(X_val_selected)

# Evaluate the new accuracy
from sklearn.metrics import accuracy_score
print(f"Optimized LightGBM Accuracy: {accuracy_score(y_val, y_pred):.4f}")


In [35]:
from sklearn.impute import SimpleImputer

# Step 1: Select only the specified columns from the test data
selected_col = numerical_col + categorical_col
test = test[selected_col]

# Step 2: Impute missing values in the selected columns
mode_imputer = SimpleImputer(strategy='most_frequent')

# Apply imputation
test[selected_col] = pd.DataFrame(
    mode_imputer.fit_transform(test[selected_col]),
    columns=selected_col,
    index=test.index
)

# Ensure proper data types after imputation
for col in numerical_col:
    test[col] = test[col].astype(float)  # Convert numerical columns to float
for col in categorical_col:
    test[col] = test[col].astype(str)    # Convert categorical columns back to string

# Step 3: Transform the test data using the preprocessing pipeline
test_transform = pipe.transform(test)

# Step 4: Apply PCA transformation to reduce dimensions
test_transform_selector = SelectKBest_selector.transform(test_transform)

# Step 5: Make predictions using the best XGBoost model
y_pred = random_search_xgb.predict(test_transform_selector)

# Step 6: If needed, map predictions back to original labels
# Example: y_pred_mapped = pd.Series(y_pred).map({0: 'no', 1: 'yes'})

# Step 7: Create a DataFrame for submission or further evaluation
prediction_df = pd.DataFrame({
    'id': test.index,   # Ensure you have an 'id' column or relevant index
    'target': y_pred       # Replace with y_pred_mapped if label mapping is applied
})

# Save the predictions to a CSV file for submission or analysis
prediction_df.to_csv('submission.csv', index=False)

# Print a sample of predictions
print(prediction_df.head())

NameError: name 'numerical_col' is not defined

In [42]:
import pandas as pd
from sklearn.impute import SimpleImputer

# Step 1: Select only the feature columns from train.csv (excluding 'target')
selected_col = [col for col in train.columns if col != 'target']  # Exclude target
test = test[selected_col]  # Select same columns from test.csv

# Step 2: Impute missing values using mode (most frequent value)
mode_imputer = SimpleImputer(strategy='most_frequent')

# Apply imputation
test = pd.DataFrame(
    mode_imputer.fit_transform(test),
    columns=selected_col,
    index=test.index
)

# Step 3: Identify numerical and categorical columns
numerical_col = test.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_col = test.select_dtypes(include=['object']).columns.tolist()

# Ensure proper data types after imputation
for col in numerical_col:
    test[col] = test[col].astype(float)  # Convert numerical columns to float
for col in categorical_col:
    test[col] = test[col].astype(str)    # Convert categorical columns back to string

# Step 4: Transform test data using the pipeline
test_transform = pipe.transform(test)

# Step 5: Apply SelectKBest for feature selection
test_transform_selector = SelectKBest_selector.transform(test_transform)

# Step 6: Make predictions using the best XGBoost model
y_pred = best_lgbm_clf.predict(test_transform_selector).astype(int)

# Step 7: Create a DataFrame for submission
prediction_df = pd.DataFrame({
    'id': test.index,  # Ensure 'id' column is present for submission
    'target': y_pred.astype(int)  # Convert predictions to integer
})

# Save the predictions to a CSV file
prediction_df.to_csv('submission.csv', index=False)

# Print a sample of predictions
print(prediction_df.head())


ValueError: could not convert string to float: 'win8defender'

In [ ]:
prediction_df = pd.DataFrame({
    'id': test.index,
    'target': y_pred
})
prediction_df.to_csv('/kaggle/working/submission.csv', index=False)